In [1]:
import numpy as np
import os
import skimage
from skimage import measure
from collections import Counter
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from radiomics import featureextractor
from radiomics.shape import RadiomicsShape

In [3]:
main_path = r'C:\Users\Wojtek\Documents\Doktorat\Astral\data\output_data'

waves = np.load(os.path.join(main_path, "waves_morph.npy"))

In [4]:
path = os.path.join(main_path, 'timespace.npy')
rel_path = os.path.join(main_path, 'segmentation_relative.h5')
abs_path = os.path.join(main_path, 'segmentation_absolute.h5')
dims_path = os.path.join(main_path, 'segmentation_dims.h5')
waves_path = os.path.join(main_path, 'waves_morph.npy')

In [14]:
timespace = np.load(path)
waves = np.load(waves_path)
rel = pd.read_hdf(rel_path)
abss = pd.read_hdf(abs_path)
dims = pd.read_hdf(dims_path)

In [15]:
dims

,id,y_min,y_max,x_min,x_max,z_min,z_max,center_y,center_x,center_z
0,0,259,345,194,268,926,1035,302,231,980
1,1,161,272,592,718,692,724,216,655,708
2,2,0,121,613,729,1134,1193,60,671,1163
3,3,210,303,257,345,852,906,256,301,879
4,4,255,397,579,692,1125,1167,326,635,1146
...,...,...,...,...,...,...,...,...,...,...
286,286,170,177,331,348,22,26,173,339,24
287,287,346,357,466,475,1158,1161,351,470,1159
288,288,332,339,297,311,17,23,335,304,20
289,289,338,349,338,348,1077,1080,343,343,1078


In [9]:
dims['id'].value_counts()

290    1
72     1
92     1
93     1
94     1
      ..
194    1
195    1
196    1
197    1
0      1
Name: id, Length: 291, dtype: int64

In [166]:
one = abss.loc[rel['id'] == 100]

In [167]:
min_x, min_y, min_z = one['x'].min(), one['y'].min(), one['z'].min()
max_x, max_y, max_z = one['x'].max(), one['y'].max(), one['z'].max()

In [168]:
segmentation = waves[min_x:max_x, min_y:max_y, min_z:max_z]
img = timespace[min_x:max_x, min_y:max_y, min_z:max_z]

In [169]:
rs = RadiomicsShape(sitk_img, sitk_mask)

In [170]:
sv_ratio = rs.getSurfaceVolumeRatioFeatureValue()
sphericity = rs.getSphericityFeatureValue()

In [163]:
segmentation = segmentation.astype('int32')
img = img.astype('uint32')

In [ ]:
# assume pixel size is in 1x1 mm. I am gonna assume 3rd dimension which is the depth is also 1 mm so that it is isotropic
data_spacing = [1,1,1]

sitk_img = sitk.GetImageFromArray(img)

sitk_mask = sitk.GetImageFromArray(segmentation)

# prepare the settings and load
params = 'settings.yaml'
extractor = featureextractor.RadiomicsFeatureExtractor(params)

#extract 
features = {}
features = extractor.execute(sitk_img, sitk_mask)

In [ ]:
sv_ratio = extractor.getSurfaceVolumeRatioFeatureValue(sitk_img, sitk_mask)

In [103]:
features

OrderedDict([('diagnostics_Versions_PyRadiomics', 'v3.0'),
             ('diagnostics_Versions_Numpy', '1.16.6'),
             ('diagnostics_Versions_SimpleITK', '1.2.4'),
             ('diagnostics_Versions_PyWavelet', '1.0.0'),
             ('diagnostics_Versions_Python', '3.7.7'),
             ('diagnostics_Configuration_Settings',
              {'minimumROIDimensions': 2,
               'minimumROISize': None,
               'normalize': False,
               'normalizeScale': 1,
               'removeOutliers': None,
               'resampledPixelSpacing': None,
               'interpolator': 'sitkBSpline',
               'preCrop': False,
               'padDistance': 5,
               'distances': [1],
               'force2D': False,
               'force2Ddimension': 0,
               'resegmentRange': None,
               'label': 1,
               'additionalInfo': True}),
             ('diagnostics_Configuration_EnabledImageTypes', {'Original': {}}),
             ('diagnost

In [38]:
import SimpleITK as sitk

In [34]:
un

array([  0, 255], dtype=uint8)

In [19]:
import radiomics

In [ ]:
img_slice = np.load('img.npy')
mask_slice = np.load('mask.npy')

# assume pixel size is in 1x1 mm. I am gonna assume 3rd dimension which is the depth is also 1 mm so that it is isotropic
data_spacing = [1,1,1]

sitk_img = sitk.GetImageFromArray(img_slice)
sitk_img.SetSpacing((float(data_spacing[0]), float(data_spacing[1]), float(data_spacing[2]) ))
sitk_img = sitk.JoinSeries(sitk_img)

sitk_mask = sitk.GetImageFromArray(mask_slice)
sitk_mask.SetSpacing((float(data_spacing[0]), float(data_spacing[1]), float(data_spacing[2]) ))
sitk_mask = sitk.JoinSeries(sitk_mask)
sitk_mask = sitk.Cast(sitk_mask, sitk.sitkInt32) # MAKE SURE IT IS CASTED IN INT

# prepare the settings and load
params = 'settings.yaml'
extractor = featureextractor.RadiomicsFeaturesExtractor(params)

#extract 
features = {}
features = extractor.execute(sitk_img, sitk_mask)

In [9]:
dims.shape[0]

291

In [10]:
rel.loc[rel['id'] == 280]

,id,y,x,z,color


In [12]:
rel['id'].value_counts()

0      163814
1      146787
2      138464
3      120849
4      105492
        ...  
251       263
252       261
253       260
254       260
255       259
Name: id, Length: 256, dtype: int64

In [39]:
waves.dtype

dtype('uint8')

In [40]:
a = measure.label(waves, connectivity=3).astype('uint16')

In [41]:
uniq, counts = np.unique(a, return_counts=True)

In [42]:
labels = uniq[1:]
counts = counts[1:]

In [43]:
sample = labels[:10]

In [44]:
def indices_label(array, label):
    return np.argwhere(array == label)

In [45]:
len(uniq)

2908

In [46]:
len(counts)

2907

In [47]:
label_counts = list(zip(labels, counts))

In [48]:
count_filtered = list(filter(lambda x: x[1] > 30, label_counts))

In [49]:
labels, counts = zip(*count_filtered)

In [50]:
inds = indices_label(a, 2)

In [51]:
from joblib import Parallel, delayed

In [52]:
b = Parallel(n_jobs=3, verbose=10)(delayed(indices_label)(a, label) for label in labels)

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:    4.0s
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:    9.1s
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:   12.0s
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:   19.5s
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:   25.0s
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:   32.8s
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:   40.7s
[Parallel(n_jobs=3)]: Done  55 tasks      | elapsed:   50.9s
[Parallel(n_jobs=3)]: Done  66 tasks      | elapsed:   59.4s
[Parallel(n_jobs=3)]: Done  79 tasks      | elapsed:  1.2min
[Parallel(n_jobs=3)]: Done  92 tasks      | elapsed:  1.4min
[Parallel(n_jobs=3)]: Done 107 tasks      | elapsed:  1.6min
[Parallel(n_jobs=3)]: Done 122 tasks      | elapsed:  1.8min
[Parallel(n_jobs=3)]: Done 139 tasks      | elapsed:  2.1min
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed:  2.3min
[Parallel(

In [53]:
with open(os.path.join(debug_path, 'waves_inds.pck'), 'wb') as file:
        pickle.dump(b, file)

In [12]:
b

[array([[0, 0, 5],
        [0, 1, 5],
        [0, 2, 5],
        [0, 3, 5],
        [1, 0, 5],
        [1, 1, 5],
        [1, 2, 5],
        [1, 3, 5],
        [2, 0, 5],
        [2, 1, 5],
        [2, 2, 5],
        [2, 3, 5],
        [3, 0, 5],
        [3, 1, 5],
        [3, 2, 5],
        [3, 3, 5]], dtype=int64),
 array([[ 0,  0, 46],
        [ 0,  1, 46],
        [ 0,  2, 46],
        [ 0,  3, 46],
        [ 1,  0, 46],
        [ 1,  1, 46],
        [ 1,  2, 46],
        [ 1,  3, 46],
        [ 2,  0, 46],
        [ 2,  1, 46],
        [ 2,  2, 46],
        [ 2,  3, 46],
        [ 3,  0, 46],
        [ 3,  1, 46],
        [ 3,  2, 46],
        [ 3,  3, 46]], dtype=int64),
 array([[ 0, 29,  3],
        [ 0, 30,  3],
        [ 0, 31,  3],
        [ 0, 32,  3],
        [ 1, 29,  3],
        [ 1, 30,  3],
        [ 1, 31,  3],
        [ 1, 32,  3],
        [ 2, 29,  3],
        [ 2, 30,  3],
        [ 2, 31,  3],
        [ 2, 32,  3],
        [ 3, 29,  3],
        [ 3, 30,  3],
      

In [13]:
import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())

In [ ]:
results = pool.map(indices_label, [label for label in sample])

In [21]:
with open(os.path.join(debug_path, 'waves_inds.pck'), 'rb') as file:
    winds = pickle.load(file)

In [17]:
counts

array([698001712,        16,        16, ...,        39,         6,
               4], dtype=int64)

In [18]:
inds = np.argwhere(a == 1)

In [36]:
label = labels[1]

In [37]:
label

2

In [42]:
output = [np.argwhere(a == label) for label in list(labels[:10])]

In [44]:
m = output[1]

In [61]:
a = a.astype('int16')

In [ ]:
results = [pool.apply(indices_label, args=(a, label)) for label in labels[:10]]

In [ ]:
labels

In [55]:
results = pool.map(np.argwhere(a == label), [label for label in labels[:10]])

TypeError: 'numpy.ndarray' object is not callable

In [19]:
inds

array([[0, 0, 5],
       [0, 1, 5],
       [0, 2, 5],
       [0, 3, 5],
       [1, 0, 5],
       [1, 1, 5],
       [1, 2, 5],
       [1, 3, 5],
       [2, 0, 5],
       [2, 1, 5],
       [2, 2, 5],
       [2, 3, 5],
       [3, 0, 5],
       [3, 1, 5],
       [3, 2, 5],
       [3, 3, 5]], dtype=int64)